In [3]:
# 基础环境检查与库安装
import sys
import os

# 定义需要安装的库
required_packages = ["pymupdf", "pytesseract", "spacy", "pandas", "openpyxl", "requests", "pillow"]

# 自动安装缺失库 (让代码更稳健，换了电脑也能跑)
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"正在安装缺失库: {package}...")
        !{sys.executable} -m pip install {package} -q

# 下载 spaCy 模型
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("正在下载 spaCy 英文模型...")
    !{sys.executable} -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")

print("✅ 所有环境配置检查完毕！")

正在安装缺失库: pillow...
✅ 所有环境配置检查完毕！


In [4]:
# Cell 2: 核心功能库导入
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
import pandas as pd
import requests
import re
from collections import Counter

pytesseract.pytesseract.tesseract_cmd = r'd:\Tesseract-OCR\tesseract.exe'

def download_sample_pdf(url, filename="sample_paper.pdf"):
    """如果本地没有文件，从网络下载一个真实的学术 PDF"""
    if os.path.exists(filename):
        print(f"📂 本地文件已存在: {filename}")
        return filename
    
    print(f"⬇️ 正在下载测试文献: {url}...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filename, 'wb') as f:
            f.write(response.content)
        print("✅ 下载完成！")
        return filename
    except Exception as e:
        print(f"❌ 下载失败: {e}")
        return None

def smart_pdf_parser(pdf_path):
    """能够同时处理文本版和扫描版 PDF 的解析器"""
    if not pdf_path: return ""
    
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"❌ 无法打开 PDF: {e}")
        return ""

    full_text = []
    print(f"🚀 开始处理文件 (共 {len(doc)} 页)...")

    for i, page in enumerate(doc):
        # 1. 尝试直接提取
        text = page.get_text()
        
        # 2. 扫描件检测逻辑：如果一页少于 50 个字，判定为图片
        if len(text.strip()) < 50:
            print(f"  ⚠️ 第 {i+1} 页疑似扫描/图表，启用 OCR 识别...")
            try:
                pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
                img = Image.open(io.BytesIO(pix.tobytes("png")))
                text = pytesseract.image_to_string(img, lang='eng')
            except Exception as e:
                print(f"  ❌ OCR 失败 (第 {i+1} 页): {e} (请确保已安装 Tesseract 软件)")
                text = "" # 失败则跳过该页
        
        full_text.append(text)
    
    return "\n".join(full_text)

# === 运行准备 ===
# 使用著名的 "Attention Is All You Need" 论文作为测试
# pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
pdf_url = "https://arxiv.org/abs/2601.09348"
local_pdf = download_sample_pdf(pdf_url, "transformer_paper.pdf")

# 执行解析
raw_content = smart_pdf_parser(local_pdf)
print(f"✅ 解析完成，共提取字符数: {len(raw_content)}")

⬇️ 正在下载测试文献: https://arxiv.org/abs/2601.09348...
✅ 下载完成！
🚀 开始处理文件 (共 11 页)...
  ⚠️ 第 9 页疑似扫描/图表，启用 OCR 识别...
  ❌ OCR 失败 (第 9 页): d:\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information. (请确保已安装 Tesseract 软件)
  ⚠️ 第 11 页疑似扫描/图表，启用 OCR 识别...
  ❌ OCR 失败 (第 11 页): d:\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information. (请确保已安装 Tesseract 软件)
✅ 解析完成，共提取字符数: 4421


In [5]:
# 文本清洗与术语挖掘 (Text Mining)
def extract_terminology(text, top_k=50):
    if not text: return pd.DataFrame()
    
    print("🧹 正在清洗文本并进行 NLP 分析...")
    
    # 清洗：修复断词 (ex- ample -> example)
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    # 清洗：合并换行符
    clean_text = text.replace('\n', ' ')
    
    # NLP 处理 (截取前 50万字符防止内存溢出)
    doc = nlp(clean_text[:500000])
    
    candidates = []
    sentences = list(doc.sents)
    
    # 提取名词短语
    for chunk in doc.noun_chunks:
        term = chunk.text.lower().strip()
        # 筛选规则：长度>2, 含空格, 无数字, 非代词开头
        if len(term) > 2 and " " in term and not any(c.isdigit() for c in term):
             if chunk[0].pos_ != "PRON":
                candidates.append(term)
    
    # 统计频率
    counts = Counter(candidates)
    
    # 构建数据表 (含例句)
    data = []
    print(f"🔍 正在为前 {top_k} 个术语匹配例句...")
    
    for term, freq in counts.most_common(top_k):
        # 查找例句
        context = "Context not found"
        for sent in sentences:
            if term in sent.text.lower():
                context = sent.text.strip().replace('\n', ' ')
                break
        
        data.append({
            "Source Term": term,
            "Frequency": freq,
            "Context (Example)": context,
            "Target Term (CN)": "" # 预留给人工填空
        })
        
    return pd.DataFrame(data)

# 执行提取
df_result = extract_terminology(raw_content)
display(df_result.head())


🧹 正在清洗文本并进行 NLP 分析...
🔍 正在为前 50 个术语匹配例句...


,Source Term,Frequency,Context (Example),Target Term (CN)
0,giulia bevilacqua,4,Title:Existence and uniqueness of minimizers f...,
1,axisymmetric nematic films,3,Title:Existence and uniqueness of minimizers f...,
2,a pdf,2,Title:Existence and uniqueness of minimizers f...,
3,the paper,2,Title:Existence and uniqueness of minimizers f...,
4,the nematic director,2,We study a variational model in which the nema...,


In [18]:
# 结果导出
output_file = "Term_Glossary.xlsx"
try:
    df_result.to_excel(output_file, index=False)
    print(f"🎉 成功！文件已保存至: {os.path.abspath(output_file)}")
    print("💡 下一步：在左侧文件栏右键点击下载，或直接使用 Excel 打开。")
except Exception as e:
    print(f"❌ 保存失败: {e}")

🎉 成功！文件已保存至: D:\20260118-Python-nlp\Term_Glossary.xlsx
💡 下一步：在左侧文件栏右键点击下载，或直接使用 Excel 打开。
